# Práctica 2

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker
import tensorflow as tf
import random
import time
import collections
import statistics

from statistics import mode
from sklearn.preprocessing import MinMaxScaler

# Quitar warnings feos
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocesamiento de datos

### Creación del dataset


In [34]:
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')
iris_OG = iris_RAW.copy()
salidas = iris_RAW.pop("class")

## Creación entradas y salidas

In [35]:
entradas = iris_RAW.to_numpy()
mm = MinMaxScaler()
entradas = mm.fit_transform(entradas)

salidas = pd.get_dummies(salidas)

# Funciones algoritmo genético

### Variables a definir

In [36]:
"""
ngenes: nneuronas de cada individuo. Sacado a traves del L4P1-EstudioRed.ipynb
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
CONVGEN: Número de generaciones que si no hay cambio para.
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
"""

ngenes = 4 # NO TOCAR
npob = 20 # NO TOCAR
ngeneraciones = 200 # TOCAR
convgen = 10 # TOCAR
pm = 0.1 # TOCAR
nres = 10 # TOCAR
nsample = 30 # TOCAR
elitismo = 0.25 # TOCAR

### Crear inidividuo

Como bien se dice en el enunciado, un individuo es un MLP sin entrenar, con tantos genes como hemos calculado. En nuestro caso son 7 genes, y el tamaño de entrada son 4. De esta manera creamos un MLP básico.

In [37]:
def crearMLP(ngenesMLP, tam_entrada,tipo,pesos):

    MLP = ker.Sequential() # Creamos el modelo vacío

    # Añadimos la primera capa (tecnicamente dos, la de entrada (con la variable input_dim) y la oculta)
    MLP.add(ker.layers.Dense(ngenesMLP, input_dim = tam_entrada, activation='relu'))

    # Añadimos la capa de salida
    MLP.add(ker.layers.Dense(3,activation='sigmoid')) 

    # Por último tenemos que configurar el modelo antes de entrenarlo con lo dicho en el enunciado
    # Optimizador: Adam
    # Función error: Mean Squared Error
    # Metricas = Binary accuracy porque nos da el número de precisión de la red
    MLP.compile(optimizer='adam',loss='mean_squared_error',metrics=["accuracy"])

    if(tipo == 1): # Le metemos pesos ya predeterminados
        MLP.set_weights(pesos)

    return MLP

### Crear población

Al igual que antes hemos creado un individuo en este caso crearemos una población. Una población es un conjunto de individuos, en nuestro caso entonces realizaremos un array de población

In [38]:
def crearPoblacion(tamPoblacion,ngenesMLP, tam_entrada):
    poblacion = []
    for i in range (tamPoblacion):
        time.sleep(1)
        individuo = crearMLP(ngenesMLP,tam_entrada,0,0)
        poblacion.append(individuo)
    return poblacion

### Evaluar población

El calculo del fitness de la población se realiza mediante el evaluate del MLP. De esta manera, simplemente tendremos que iterar toda la población y devolver un array con el fitness de cada individuo en la posición de la población.

In [39]:
def evaluarPoblacion(poblacion,entradas_test,esperadas_test):
    evalPobl = []
    for individuo in poblacion:
        metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
        evalPobl.append(metricas_totales*150)
    return evalPobl

### Evaluar individuo

Adicionalmente por si es requerido, crearemos una evaluacion para un unico individuo

In [40]:
def evaluarIndividuo(individuo,entradas_test,esperadas_test):
    metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
    evalInd = metricas_totales*150
    return evalInd

### Selección de replicar individuo

Posteriormente necesitaremos un algoritmo de montecarlo, que nos permita elegir la probablididad de selección de el individuo. El método de montecarlo es generar un número aleatorio entre [0,1], si este número es menor que la probabilidad de selección, el individuo se replica, si no, no. En nuestro caso, recibimos una población y la recorremos, viendo cuales se cogen o no. De esta manera, devolvemos un de booleano donde indicamos si ese individuo se replica o no.

In [41]:
def seleccionados(poblacion,entradas_test,esperadas_test):
    fis_totales = evaluarPoblacion(poblacion,entradas_test,esperadas_test)
    pob_fis = dict(zip(poblacion,fis_totales))

    pob_fis_ordenado = {k: v for k, v in sorted(pob_fis.items(), key=lambda item: item[1], reverse=True)}
    nElegidos = int(npob * elitismo) # X % pobabilidad dependiendo del valor elitismo
    mejores = []
    for individuo in pob_fis_ordenado:
        mejores.append(individuo)
        nElegidos -= 1
        if(nElegidos == 0):
            break
    return mejores

### Mutar

Secuencialmente para todos los genes del padre decide si cada gen se copia fielmente en el hijo o se muta usando un Método de Montecarlo simple sobre el valor de Pm. Si muta, cámbialo por otro valor aleatorio en el rango (-1, 1). Prueba también otras opciones de mutación. 

In [42]:
def mutar(individuo):
    pesos_antiguos = individuo.get_weights()
    pesos_entrada_oculta = pesos_antiguos[0]
    pesos_oculta_salida = pesos_antiguos[2]

    for i in range(len(pesos_entrada_oculta)):
        for j in range(len(pesos_entrada_oculta[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_entrada_oculta[i][j] = random.uniform(-1, 1)

    for i in range(len(pesos_oculta_salida)):
        for j in range(len(pesos_oculta_salida[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_oculta_salida[i][j] = random.uniform(-1, 1)
    
    aux1 = np.zeros(ngenes, dtype='float32')
    aux2 = np.array([0.,0.,0.], dtype='float32')
    pesos_nuevos = [pesos_entrada_oculta,aux1,pesos_oculta_salida,aux2] # Los pesos con mutación

    nuevoMLP = crearMLP(ngenes,4,1,pesos_nuevos)

    return nuevoMLP

### Cromosoma consenso

El cromosoma consenso es aquel individuo que tiene los genes más repetidos. De esta manera de cada generación podríamos saber un individuo medio, teniendo las cualidades más típicas de la generación

In [43]:
def cromosomaconsenso(poblacion, ngenes):
    matriz1_cro = np.empty((4, ngenes))
    matriz2_cro = np.empty((ngenes, 3))

    for i in range(len(matriz1_cro)):
        for j in range(len(matriz1_cro[i])):
            lista_pos = []
            for individuo in poblacion:
                pesos_antiguos_individuo = individuo.get_weights()
                pesos_ind = pesos_antiguos_individuo[0]
                lista_pos.append(pesos_ind[i][j])
            matriz1_cro[i][j] = mode(lista_pos)
            print(lista_pos)
            print(matriz1_cro[i][j])

    for i in range(len(matriz2_cro)):
        for j in range(len(matriz2_cro[i])):
            lista_pos = []
            for individuo in poblacion:
                pesos_antiguos_individuo = individuo.get_weights()
                pesos_ind = pesos_antiguos_individuo[2]
                lista_pos.append(pesos_ind[i][j])
            matriz2_cro[i][j] = mode(lista_pos)
    
    aux1 = np.zeros(ngenes, dtype='float32')
    aux2 = np.array([0.,0.,0.], dtype='float32')
    pesos_nuevos = [matriz1_cro,aux1,matriz2_cro,aux2] # Los pesos con mutación

    cromosomaCom = crearMLP(ngenes,4,1,pesos_nuevos)

    return cromosomaCom

### Resumen

In [44]:
def resumen(poblacion,entradas,salidas,generacion):
    pobFit = evaluarPoblacion(poblacion,entradas,salidas)
    valMejInd = np.amax(pobFit)
    nInd = collections.Counter(pobFit)[valMejInd]   
    return valMejInd,str((nInd/npob)*100)

### Simulación

In [45]:
def simular():

    fitantiguo = 0
    fitnuevo = 0
    
    poblacion = crearPoblacion(npob,ngenes, 4) # Población inicial

    ultimaGen = 0 # Numero de generacion necesaria para obtener el resultado optimo 
    ultimoFitness = 0 # Mejor fitness posible con esta configuracion de
    ultimoPorcMejoresInd = 0 # Ultimo porcentaje de mejores individuos

    fitnes_antiguo = 0
    num_conv = 0 # numero de veces que el fitnes se ha repetido de
    lim_conv = 5 # limite de veces que el fitnes se puede repetir para la condicion de terminacion                              TOCAR

    for i in range(ngeneraciones):
        nueva_poblacion = []
        
        fitnes = 0
        porcentaje = 0

        mejoresIndividuos = seleccionados(poblacion,entradas,salidas)

        for indvidio_mutar in mejoresIndividuos:
            for x in range(int(1/elitismo)):
                nuevoIndividuo = mutar(indvidio_mutar) # Lo mutamos
                nueva_poblacion.append(nuevoIndividuo)
        
        fitnes, porcentaje = resumen(nueva_poblacion,entradas,salidas,i) # esto se hace siempre, en cada una de las generaciones
        print(f"Generacion {str(i+1)} -> fitnes {fitnes}")
        ultimoFitness = fitnes
        ultimoPorcMejoresInd = porcentaje
        if(fitnes_antiguo == fitnes):
            num_conv += 1
        else:
            num_conv = 0
            fitnes_antiguo = fitnes

        poblacion = nueva_poblacion.copy()

        # Condicion de parada
        
        ultimaGen = i
        if(num_conv == lim_conv):
            return [ultimaGen,ultimoFitness, ultimoPorcMejoresInd]
        
    return [ultimaGen, ultimoFitness, ultimoPorcMejoresInd]

## Comienza el automatizador
</br>Primero inicializamos las variables y creamos las listas con los valores que queremos probar

In [46]:
ngeness = [i for i in range(0,11,2) if i != 0] # AUTOMATIZAR
npobs = [i for i in range(0,51,10) if i != 0] # AUTOMATIZAR
ngeneraciones = 200 # TOCAR
convgen = 10 # TOCAR
pms = [0.01,0.05,0.1,0.15,0.2,0.25,0.3] # AUTOMATIZAR
nres = 10 # TOCAR
nsample = 30 # TOCAR
elitismos = [5,10,20,25,50] # TOCAR

Y ahora realizamos los bucles anidados:

In [47]:
salidaOG = [0,0,0] # guarda el numero de generaciones, el fitnes y el porcentaje individuos con ese fitnes del mejor algoritmo
solucion = [] # guarda que valor de npob y pm corresponde con salidaOG
for i in ngeness:
    for j in npobs:
        print("=================================================================================")
        print(f"Probando con el numero de genes = {i} y el numero de poblaciones = {j}")
        print("=================================================================================")
        # Ahora vamos a realizar el algorimo genético con los valores dados y vemos que nos da para compararlo
        ngnenes = i 
        npob = j
        salidaN = simular() # guardamos los valores en la salida Nueva
        if salidaOG[1] < salidaN[1]: # 1 -> fitnes del mejor individuo  
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]}")
            salidaOG = salidaN
            solucion = [i,j]
        elif salidaOG[1] == salidaN[1]: # 0 -> Numero de generaciones
            if salidaOG[0] > salidaN[0]: # si el fitnes es igual, pero la nueva salida lo ha conseguido con menos generaciones, esta sera la nueva mejor
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el numero de generaciones, Antiguo: {salidaN[0]} Nuevo: {salidaOG[0]}")
                salidaOG = salidaN
                solucion = [i,j]
        elif salidaOG[0] == salidaN[0] and salidaOG[1] == salidaN[1]: # si ambos valores son iguales, se busca aquel con mayor porcentaje de elementos con el mejor fitness
            if salidaOG[2] < salidaN[2]: #2 -> % de individuos con mejor fitness
                print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el % de individuos con mejor fitnes, Antiguo: {salidaN[2]} Nuevo: {salidaOG[2]}")
                salidaOG = salidaN
                solucion = [i,j]
print(f"La combinacion perfecta encontrada es de numero de genes = {solucion[0]} y el numero de poblaciones = {solucion[1]}")
print(f"Dando como resultado un fitnes de {salidaOG[1]}/150, con un numero de generaciones de {salidaOG[0]} y un porcentaje de individuos con ese fitnes de {salidaOG[2]}")

Probando con el numero de genes = 2 y el numero de poblaciones = 10
Generacion 1 -> fitnes 88.05
Generacion 2 -> fitnes 106.05
Generacion 3 -> fitnes 106.05
Generacion 4 -> fitnes 100.05000000000001
Generacion 5 -> fitnes 100.05000000000001
Generacion 6 -> fitnes 100.05000000000001
Generacion 7 -> fitnes 100.05000000000001
Generacion 8 -> fitnes 100.05000000000001
Generacion 9 -> fitnes 100.05000000000001
El fitnes nuevo: 100.05000000000001 ha sustituido al antiguo: 0
Probando con el numero de genes = 2 y el numero de poblaciones = 20


KeyboardInterrupt: 

In [ ]:
print("Ahora buscamos la relacion entre esos valores y la probilidad de mutacion")
solucion.append(pm)
for i in pms:
    pm = i
    ngenes = 6 #solucion[0]
    npob = 50 # solucion[1]
    print("=========================================================")
    print(f"Probando con la probabilidad de mutacion = {i}")
    print("=========================================================")
    salidaN = simular() # guardamos los valores en la salida Nueva
    if salidaOG[1] < salidaN[1]: # 1 -> fitnes del mejor individuo  
        print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]}")
        salidaOG = salidaN
        solucion = [solucion[0],solucion[1],i]
    elif salidaOG[1] == salidaN[1]: # 0 -> Numero de generaciones
        if salidaOG[0] > salidaN[0]: # si el fitnes es igual, pero la nueva salida lo ha conseguido con menos generaciones, esta sera la nueva mejor
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el numero de generaciones, Antiguo: {salidaN[0]} Nuevo: {salidaOG[0]}")
            salidaOG = salidaN
            solucion = [solucion[0],solucion[1],i]
    elif salidaOG[0] == salidaN[0] and salidaOG[1] == salidaN[1]: # si ambos valores son iguales, se busca aquel con mayor porcentaje de elementos con el mejor fitness
        if salidaOG[2] < salidaN[2]: #2 -> % de individuos con mejor fitness
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el % de individuos con mejor fitnes, Antiguo: {salidaN[2]} Nuevo: {salidaOG[2]}")
            salidaOG = salidaN
            solucion = [solucion[0],solucion[1],i]
print(f"Por lo tanto, la red ejecutada que mejor ha funcionado tiene:\n Nº de genes = {solucion[0]}\n Nº de poblaciones = {solucion[1]}\n Probabilidad de mutacion = {solucion[2]}")
print(f"Dando como resultado un fitnes de {salidaOG[1]}/150, con un numero de generaciones de {salidaOG[0]} y un porcentaje de individuos con ese fitnes de {salidaOG[2]}")

IndentationError: unexpected indent (Temp/ipykernel_22380/1528632517.py, line 11)

In [ ]:
print(f"Y ahora, vemos el porcentaje de elitismo perfecto para estos valores")
solucion.append(elitismo)
for i in elitismos:
    elitismo = i 
    pm = solucion[2]
    ngenes = solucion[0]
    npob = solucion[1]
    print("=========================================================")
    print(f"Probando con el elitismo = {i}")
    print("=========================================================")
    salidaN = simular() # guardamos los valores en la salida Nueva
    if salidaOG[1] < salidaN[1]: # 1 -> fitnes del mejor individuo  
        print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]}")
        salidaOG = salidaN
        solucion = [solucion[0],solucion[1],solucion[2],i]
    elif salidaOG[1] == salidaN[1]: # 0 -> Numero de generaciones
        if salidaOG[0] > salidaN[0]: # si el fitnes es igual, pero la nueva salida lo ha conseguido con menos generaciones, esta sera la nueva mejor
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el numero de generaciones, Antiguo: {salidaN[0]} Nuevo: {salidaOG[0]}")
            salidaOG = salidaN
            solucion = [solucion[0],solucion[1],solucion[2],i]
    elif salidaOG[0] == salidaN[0] and salidaOG[1] == salidaN[1]: # si ambos valores son iguales, se busca aquel con mayor porcentaje de elementos con el mejor fitness
        if salidaOG[2] < salidaN[2]: #2 -> % de individuos con mejor fitness
            print(f"El fitnes nuevo: {salidaN[1]} ha sustituido al antiguo: {salidaOG[1]} por el % de individuos con mejor fitnes, Antiguo: {salidaN[2]} Nuevo: {salidaOG[2]}")
            salidaOG = salidaN
            solucion = [solucion[0],solucion[1],solucion[2],i]
print(f"Por lo tanto, la red ejecutada que mejor ha funcionado tiene:\n Nº de genes = {solucion[0]}\n Nº de poblaciones = {solucion[1]}\n Probabilidad de mutacion = {solucion[2]}\n Elitismo = {solucion[3]}")
print(f"Dando como resultado un fitnes de {salidaOG[1]}/150, con un numero de generaciones de {salidaOG[0]} y un porcentaje de individuos con ese fitnes de {salidaOG[2]}")